<a href="https://colab.research.google.com/github/jreagle/jreagle/blob/main/dev/Maintenance_(CBA)_Retrofit_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install dateparser
!pip install slugify

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for slugify: filename=slugify-0.0.1-py3-none-any.whl size=1883 sha256=9bbd7fcad43f6dcd159829523e7b2748ba03aeca65bfda5be7ad558f9e205358
  Stored in directory: /root/.cache/pip/wheels/6d/64/09/b81b14a116496868ebac7150f737a26b11b140bfa44f555fcf
Successfully built slugify


In [ ]:
#@title Folder Base (Google Drive)
folder_base = '/content/drive/MyDrive/Share - Elias FNOP/Pessoal (FNOP)/Finanças & Bens/FolkDigital (FNOP)/CBA/Retrofit2025'

In [ ]:
#@title Global/Utils Funtions
from bs4 import BeautifulSoup as bs
from PIL import ImageFile
import datetime as dt
import pytz


def getSizeImageURL(url):
    # Source: https://stackoverflow.com/questions/7460218/get-image-size-without-downloading-it-in-python?answertab=trending#tab-top

    resume_header = {'Range': 'bytes=0-2000000'}    ## the amount of bytes you will download
    data = requests.get(url, stream = True, headers = resume_header).content
    p = ImageFile.Parser()
    p.feed(data)    ## feed the data to image parser to get photo info from data headers
    if p.image:
        # print(p.image.size) ## get the image size (Width, Height)
        ## output: (1400, 1536)
        return p.image.size

# getSizeImageURL('https://www.cezarbritto.adv.br/img/logo_congressoemfoco.jpg')

def staticSEO(
        page_slug:str,
        page_name:str,
        page_description:str,
        page_datetime:str, # article:published_time => 2023-02-28T22:00:00+0000
        page_image_url:str,
        page_image_width:int,
        page_image_height:int,
        page_section:str='',
        page_type:str='website', # og:type => [website | article]
        ):
    dt_pub = dtparser.parse(page_datetime)
    if dt_pub.tzinfo is None or dt_pub.tzinfo.utcoffset(dt_pub) is None:
        page_datetime = dt_pub.strftime("%Y-%m-%dT%H:%M:%S-0300")
    dt_now = dt.datetime.now(pytz.timezone('America/Sao_Paulo')).strftime("%Y-%m-%dT%H:%M:%S%z")
    seo_tags = f'''
    <!-- This site is optimized with the Static SEO Tool -->
    <link href="https://www.cezarbritto.adv.br/{page_slug}" rel="canonical">
    <meta content="{page_description}" name="description">
    <meta content="pt_BR" property="og:locale">
    <meta content="{page_type}" property="og:type">
    <meta content="{page_name} | Cezar Britto Advocacia - CBA" property="og:title">
    <meta content="{page_description}" property="og:description">
    <meta content="https://www.cezarbritto.adv.br/{page_slug}" property="og:url">
    <meta content="{dt_now}" property="og:updated_time">
    <meta content="{page_image_url}" property="og:image">
    <meta content="{page_image_width}" property="og:image:width">
    <meta content="{page_image_height}" property="og:image:height">

    <meta content="summary" name="twitter:card">
    <meta content="{page_name} | Cezar Britto Advocacia - CBA" name="twitter:title">
    <meta content="{page_description}" name="twitter:description">
    <meta content="@cezarbritto_cba" name="twitter:site">
    <meta content="{page_image_url}" name="twitter:image">
    <meta content="@cezarbritto_cba" name="twitter:creator">

    '''

    if page_type == 'article':
        tags_article = f'''
        <meta content="{page_section}" property="article:section">
        <meta content="{page_datetime}" property="article:published_time">
        <meta content="{dt_now}" property="article:modified_time">
        <meta content="https://www.facebook.com/cezarbrittoadvogados" property="article:publisher">
        '''
        seo_tags = seo_tags+tags_article

    return bs(seo_tags,'html.parser')

In [ ]:
#@title Scrape Old SIte (News/Articles)
import requests
from bs4 import BeautifulSoup as bs
from slugify import slugify
import dateparser as dtparser
from pprint import pprint
import json

def scrapePosts():
    allposts = requests.get('https://www.cezarbritto.adv.br/data/allposts.json').json()
    publications = {
        'news':[],
        'articles':[]
        }
    for cat in list(publications.keys()):
        for idx_post,post in enumerate(allposts[cat]):
            post_resp = requests.get(f"https://www.cezarbritto.adv.br/{post['filename']}.html")
            post_soup = bs(post_resp.content,'html.parser')
            author_span = post_soup.select_one('.content-post .meta-post .meta-post-left span').text
            fig_caption = post_soup.select_one('.content-post figure figcaption')
            pub_item = {
                'slug': post['filename'],
                'author': [{slugify(author_span):author_span}],
                'datetime': dtparser.parse(post['postDateTXT']).strftime('%Y-%m-%dT%H:%M:%S'),
                'image': [{
                    'href': post['imgfeatured'],
                    'extension': post['imgfeatured'].split('.')[-1],
                    'caption': "" if fig_caption == None  else fig_caption.text,
                    'alt': "" if fig_caption == None  else fig_caption.text,
                    'size': getSizeImageURL(post['imgfeatured'])
                }],
                'title': post['postTitle'],
                'excerpt': post['postExcerpt'],
                'html_content': str(post_soup.select_one('.content-post .post')).replace('<div class="post"','<div class="pub-container"')
            }
            print(f"{cat} ({idx_post}) => {pub_item}")
            publications[cat].append(pub_item)


    out_json = json.dumps(publications,indent=4)
    print(out_json)

    with open(f"{folder_base}/data/publications.json",'w') as new_file:
        new_file.write(out_json)

In [ ]:
#@title Create => Blog Post [News/Articles]
from bs4 import BeautifulSoup as bs
from ipywidgets import HTML

def newBlogPost(pubCat:str,pubItem:dict,prod:bool=False):
    # BASE STYLE => INLINE
    # css_fonts = open(f"{folder_base}/css/preload_fonts.css").read()
    # css_retrofit = open(f"{folder_base}/css/retrofit2024.css").read()
    # css_retrofit = bs(f'''<style id="cba-global">{css_retrofit}</style>''','html.parser')
    # base_soup.select_one('style#cba-global').replace_with(css_retrofit)
    # BASE EXTEND => MAIN (BLOG)
    base_soup = bs(open(f"{folder_base}/templates/base.html").read(),'html.parser')
    blog_soup = bs(open(f"{folder_base}/templates/blog.html").read(),'html.parser')
    month_names = ['Janeiro', 'Fevereiro', 'Março', 'Abril', 'Maio', 'Junho', 'Julho', 'Agosto', 'Setembro', 'Outubro', 'Novembro', 'Dezembro']
    dt_friendly = f"{pubItem['datetime'][8:10]} de {month_names[int(pubItem['datetime'][5:7])-1]} de {pubItem['datetime'][:4]}"
    [author_slug,author_string] = list(*pubItem['author'][0].items())
    blog_soup.select_one('main .section-header h1').string = pubItem['title']
    blog_soup.select_one('main .blog-post-meta-left img')['src'] = f"https://www.cezarbritto.adv.br/img/{author_slug}.png"
    blog_soup.select_one('main .blog-post-meta-left span').string = author_string
    blog_soup.select_one('main .blog-post-meta-right span').string = dt_friendly
    blog_soup.select_one('main .blog-post-featured img')['src'] = pubItem['image'][0]['href']
    blog_soup.select_one('main .blog-post-featured img')['alt'] = pubItem['image'][0]['alt']
    blog_soup.select_one('main .blog-post-featured figcaption').string = pubItem['image'][0]['caption']
    blog_soup.select_one('main .blog-post-content').clear()
    blog_soup.select_one('main .blog-post-content').append(bs(pubItem['html_content'],'html.parser'))
    # BASE OUTPUT
    base_soup.select_one('main').replace_with(blog_soup.select_one('main'))
    base_soup.select_one('title').insert_after(staticSEO(
        page_type='article',
        page_section=pubCat,
        page_name=pubItem['title'],
        page_slug=pubItem['slug'],
        page_description=pubItem['excerpt'],
        page_datetime=pubItem['datetime'],
        page_image_url=pubItem['image'][0]['href'],
        page_image_width=pubItem['image'][0]['size'][0],
        page_image_height=pubItem['image'][0]['size'][1]
        ))
    base_soup.title.string = pubItem['title'] + " | Cezar Britto Advocacia - CBA"


    if prod:
        with open(f"{folder_base}/pages/publications/{pubCat}/{pubItem['slug']}.html",'w') as new_file:
            new_file.write(str(base_soup))
    else:
        display(HTML(str(base_soup)))

# newBlogPost('news',pub_item)


In [ ]:
with open(f"{folder_base}/data/publications.json",'r') as file:
    publications = json.loads(file.read())

if True:
    for cat in ['news','articles']:
        for pubIdx,pubItem in enumerate(publications[cat]):
            print(cat,f'({pubIdx+1}) =>',pubItem['slug'])
            newBlogPost(cat,pubItem,prod=True)

news (1) => audiencia-aprofunda-debate-sobre-uberizacao-mas-ha-divergencia-de-dados
news (2) => concessao-de-reducao-da-jornada-de-trabalho-em-50-e-autorizacao-do-exercicio-da-atividade-em-regime-de-teletrabalho-integral-de-pessoa-com-deficiencia
news (3) => reconhecimento-de-licitude-de-acumulacao-dos-cargos-de-tecnico-judiciario-com-o-de-professor
news (4) => tst-reforma-decisao-do-trt-para-deferir-horas-extras-e-reflexos-com-base-no-divisor-200
news (5) => tst-majora-valor-da-condenacao-em-danos-morais-decorrente-de-acidente-de-trabalho
news (6) => vitoria-reconhecimento-de-servidora-portadora-de-transtorno-do-espectro-autista-enquanto-pessoa-com-deficiencia
news (7) => cfj-confirma-decisao-sobre-absorcao-de-quintos-decimos-vitoria-para-a-administracao-publica
news (8) => justica-determina-que-camara-de-sp-so-vote-venda-da-sabesp-apos-todas-as-audiencias
news (9) => tst-reconhece-a-invalidade-do-pedido-de-demissao-de-funcionaria-gravida
news (10) => tjpr-reforma-decisao-para-determi

In [ ]:
#@title Create => Expertise Pages (Sub)

with open(f"{folder_base}/data/expertises.json",'r') as file:
    expertises = json.loads(file.read())

def makePageExpertise(slugItem,dictItem,prod=False):
    img_src = f"https://www.cezarbritto.adv.br/img/featured_{slugItem.replace('-','_')}.png"
    img_size = getSizeImageURL(f"https://www.cezarbritto.adv.br/retrofit/img/featured_{slugItem.replace('-','_')}.png")
    main_html = f'''
        <main id="{slugItem}" class="subpage">
        <section class="expertise hero">
            <h1>{dictItem['title']}</h1>
            <img src="{img_src}" alt="Capa {dictItem['title']}">
        </section>
        <section class="expertise content">
            <div class="container">
                <div class="container_left expertise_paragraphs">{''.join(dictItem['paragraphs'])}</div>
                <div class="container_right">
                    <b>
                        <h4>DENTRE AS ATIVIDADES, DESTACAM-SE:</h4>
                    </b>
                    <br>
                    <ul class="expertise_activities">{''.join(dictItem['activities'])}</ul>
                </div>
            </div>
        </section>
        <section id="insights">
            <h2>CONTEÚDOS DIGITAIS</h2>
            <div class="filter-bar">
                <ul>
                    <li onclick="filterBar(this)" class="active" value="news">Notícias</li>
                    <li onclick="filterBar(this)" class="" value="articles">Artigos</li>
                    <li onclick="filterBar(this)" class="" value="videos">Vídeos</li>
                </ul>
            </div>
            <div class="last-insights-container">
                <div class="last-insights-group" cat="news"></div>
                <div class="last-insights-group" cat="articles" style="display:none;"></div>
                <div class="last-insights-group" cat="videos" style="display:none;"></div>
            </div>
        </section>
    </main>
    '''
    # BASE OUTPUT
    base_soup = bs(open(f"{folder_base}/templates/base.html").read(),'html.parser')
    base_soup.select_one('main').replace_with(bs(main_html,'html.parser'))
    base_soup.select_one('title').insert_after(staticSEO(
        page_type='website',
        # page_section=pubCat,
        page_name=dictItem['title'],
        page_slug=slugItem,
        page_description= f"Área {dictItem['title']}",
        page_datetime="2017-01-20T00:00:00-0300",
        page_image_url=img_src,
        page_image_width=img_size[0],
        page_image_height=img_size[1]
        ))
    base_soup.title.string = f"Área {dictItem['title']} | Cezar Britto Advocacia - CBA"


    if prod:
        with open(f"{folder_base}/pages/expertises/{slugItem}.html",'w') as new_file:
            new_file.write(str(base_soup))
    else:
        display(HTML(str(base_soup)))

# makePageExpertise('area-controladoria',m['area-controladoria'],prod=True)

In [ ]:
for slugItem,dictItem in expertises.items():
    # print(dictItem)
    makePageExpertise(slugItem=slugItem,dictItem=dictItem,prod=True)

In [ ]:
o = {'j':"<li>Cursando pós-graduação em Direito Público com ênfase em Direito Administrativo para Servidores Públicos Federais pela Proordem.</li>"}
print(json.dumps(o))

{"j": "<li>Cursando p\u00f3s-gradua\u00e7\u00e3o em Direito P\u00fablico com \u00eanfase em Direito Administrativo para Servidores P\u00fablicos Federais pela Proordem.</li>"}


In [ ]:
#@title Create => Repository Pulications [News/Articles]

def makePageRepository(dataCat,dataList,prod=None):
    contaneiner_publist = []
    for pubIdx,pubItem in enumerate(dataList):
        box_pub = f'''
        <div class="box-post" {'style="display:none"' if pubIdx > 9 else ""}>
            <div class="thumb-post">
                <a aria-label="Noticia na integra" href="{pubItem['slug']}" target="_blank">
                    <img alt="Miniatura da Imagem de Destaque da Noticia" src="{pubItem['image'][0]['href']}">
                </a>
            </div>
            <div class="info-post">
                <a aria-label="Noticia na integra" class="title" href="{pubItem['slug']}" target="_blank">
                    <h2>{pubItem['title']}</h2>
                </a>
                <p>{pubItem['excerpt'][:123]}...</p>
                <a aria-label="Noticia na integra" class="null"
                    href="{pubItem['slug']}"
                    target="_blank">
                    <button class="btn-moss">Continue lendo...</button>
                </a>
            </div>
        </div>
        '''
        # print(box_pub)
        contaneiner_publist.append(box_pub)

    meta = {
        "news":['Notícias','noticias'],
        "articles":["Artigos", "artigos"],
        "videos": ["Videos","artigos"]
    }

    main_html = f'''
    <main class="subpage">
        <section class="section-header">
            <h1 class="subpage-title blog">{meta[dataCat][0]}</h1>
        </section>
        <section class="repository">
                <div class="widget-search">
                    <input name="pesquisa" type="text">
                    <button>Pesquisar</button>
                </div>
                <div class="container-postlist">{''.join(contaneiner_publist)}</div>
        </section>
    </main>
    '''

    base_soup = bs(open(f"{folder_base}/templates/base.html").read(),'html.parser')
    base_soup.select_one('main').replace_with(bs(main_html,'html.parser'))



    base_soup.select_one('title').insert_after(staticSEO(
        page_type='website',
        # page_section=pubCat,
        page_name=meta[dataCat][0],
        page_slug=meta[dataCat][1],
        page_description= f"",
        page_datetime="2017-01-20T00:00:00-0300",
        page_image_url="",
        page_image_width=0,
        page_image_height=0
        ))
    base_soup.title.string = f"{meta[dataCat][0]} | Cezar Britto Advocacia - CBA"


    if prod:
        with open(f"{folder_base}/pages/{dataCat}.html",'w') as new_file:
            new_file.write(str(base_soup))
    else:
        # BASE STYLE => INLINE
        # css_fonts = open(f"{folder_base}/css/preload_fonts.css").read()
        css_retrofit = open(f"{folder_base}/css/retrofit2024.css").read()
        css_retrofit = bs(f'''<style id="cba-global">{css_retrofit}</style>''','html.parser')
        base_soup.select_one('style#cba-global').replace_with(css_retrofit)
        display(HTML(str(base_soup)))

makePageRepository('news',publications['news'],prod=True)